**Importing Dependencies and Dataset**

In [1]:
import pandas as pd
from transformers import pipeline
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from math import exp
from tqdm import tqdm 
import numpy as np 
import pandas as pd 
import os


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/mydata-csv/train.csv


**Lading data and selecting pretranied models**

In [2]:
# Load Data
train_df_ = pd.read_csv("/kaggle/input/mydata-csv/train.csv")
train_df = train_df_.head(150)

# Select Pretrained Models
models = [
    "facebook/bart-large-cnn",
    "t5-large",
    "sshleifer/distilbart-cnn-12-6",
    "google/pegasus-large",
    "allenai/led-large-16384-arxiv",
]

**Calculating BLEU Score, Semantic Coherence, Factual Accuracy and Content Coverage to compare the above 5 models**

In [3]:
# Initialize BLEU Scores list
bleu_scores = []

# Semantic Coherence (Example implementation)
def semantic_coherence(generated_summary, dialogue):
    # Your semantic coherence metric calculation logic
    summary_tokens = word_tokenize(generated_summary.lower())
    dialogue_tokens = word_tokenize(dialogue.lower())

    # Calculate the intersection of tokens
    common_tokens = set(summary_tokens) & set(dialogue_tokens)

    # Calculate semantic coherence score based on the ratio of common tokens to summary length
    coherence_score = len(common_tokens) / len(summary_tokens)

    return coherence_score

# Factual Accuracy (Example implementation)
def factual_accuracy(generated_summary, reference_summary):
    # Your factual accuracy metric calculation logic
    gen_tokens = set(word_tokenize(generated_summary.lower()))
    ref_tokens = set(word_tokenize(reference_summary.lower()))

    # Calculate the intersection of tokens
    common_tokens = gen_tokens & ref_tokens

    # Calculate factual accuracy score based on the ratio of common tokens to reference summary length
    accuracy_score = len(common_tokens) / len(ref_tokens) if len(ref_tokens) != 0 else 0
    return accuracy_score

# Content Coverage (Example implementation)
def content_coverage(generated_summary, dialogue):
    # Your content coverage metric calculation logic
    
    summary_tokens = set(word_tokenize(generated_summary.lower()))
    dialogue_tokens = set(word_tokenize(dialogue.lower()))

    # Calculate the intersection of tokens
    common_tokens = summary_tokens & dialogue_tokens

    # Calculate the content coverage score based on the ratio of common tokens to dialogue length
    coverage_score = len(common_tokens) / len(dialogue_tokens) if len(dialogue_tokens) != 0 else 0
    return coverage_score

# Initialize evaluation results DataFrame
evaluation_results = pd.DataFrame(columns=["Model", "BLEU Score", "Semantic Coherence", "Factual Accuracy", "Content Coverage"])

# Initialize empty list to store evaluation results
evaluation_results_list = []

# Apply Models and Evaluate
for model_name in models:
    print(f"Evaluating model: {model_name}")
    
    # Initialize the summarization pipeline
    summarizer = pipeline("summarization", model=model_name, tokenizer=model_name)
    
    # Initialize evaluation metric accumulators
    semantic_coherence_scores = []
    factual_accuracy_scores = []
    content_coverage_scores = []
    
    # Generate summaries
    generated_summaries = []
    for index, row in tqdm(train_df.iterrows(), total=len(train_df)):  # Use tqdm to show progress
        dialogue = row['dialogue']
        summary = row['summary']
        generated_summary = summarizer(dialogue, max_length=100, min_length=30, do_sample=False)[0]["summary_text"]
        generated_summaries.append(generated_summary)
        
        # Evaluate Semantic Coherence
        coherence_score = semantic_coherence(generated_summary, dialogue)
        semantic_coherence_scores.append(coherence_score)
        
        # Evaluate Factual Accuracy
        accuracy_score = factual_accuracy(generated_summary, summary)
        factual_accuracy_scores.append(accuracy_score)
        
        # Evaluate Content Coverage
        coverage_score = content_coverage(generated_summary, dialogue)
        content_coverage_scores.append(coverage_score)

    
    # Calculate BLEU Score
    reference_summaries = train_df["summary"].tolist()
    bleu_score = corpus_bleu([[summary] for summary in reference_summaries], generated_summaries)
    bleu_scores.append(bleu_score)
    
    # Append results to the evaluation results list
    evaluation_results_list.append({
        "Model": model_name,
        "BLEU Score": bleu_score,
        "Semantic Coherence": sum(semantic_coherence_scores) / len(semantic_coherence_scores),
        "Factual Accuracy": sum(factual_accuracy_scores) / len(factual_accuracy_scores),
        "Content Coverage": sum(content_coverage_scores) / len(content_coverage_scores),
    })
    
    # Print a separator for clarity
    print("="*50)

# Concatenate the evaluation results list into a DataFrame
evaluation_results = pd.concat([pd.DataFrame(item, index=[0]) for item in evaluation_results_list], ignore_index=True)

# Compare Results
print("BLEU Scores:", bleu_scores)



Evaluating model: facebook/bart-large-cnn


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

100%|██████████| 150/150 [36:52<00:00, 14.75s/it]

BLEU Scores: [0.3488121983156154, 0.3397876196809291, 0.30525945503925905, 0.36171531600300605, 0.15297269719096934]


**Saving results to csv, which will be used as input for calculating topsis score for each model**


In [5]:
# Save evaluation results to a CSV file
evaluation_results.to_csv("inputfortopsis.csv", index=False)

In [7]:
evaluation_results.head()

,Model,BLEU Score,Semantic Coherence,Factual Accuracy,Content Coverage
0,facebook/bart-large-cnn,0.348812,0.655736,0.458349,0.371619
1,t5-large,0.339788,0.755254,0.457265,0.408908
2,sshleifer/distilbart-cnn-12-6,0.305259,0.708327,0.521721,0.483919
3,google/pegasus-large,0.361715,0.707104,0.420562,0.363004
4,allenai/led-large-16384-arxiv,0.152973,0.243868,0.284099,0.186362
